In [49]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pickle

In [50]:
#import datasets
df = pd.read_csv('./plataformas_ratings.csv', sep=",")
df = df.reset_index()
df.shape


(22998, 17)

In [51]:
df= df.sample(n=1000, random_state=38)
df.shape

(1000, 17)

In [52]:
df.head()

,index,id,show_id,type,title,director,cast,country,date_added,release_year,rating_x,listed_in,description,duration_int,duration_type,Plataforma,score_mean
3854,3854,as3855,s3855,movie,happy birthday,casey tebo,"tristin mays, vanessa lengies, erik palladino",NaN,NaN,2016,r,"comedy, suspense",when two friends embark on a journey to mexica...,90.0,min,amazon,3.57
13542,13542,hs2425,s2425,tv show,chopped junior,NaN,NaN,united states,2018-09-13,2015,tv-g,"cooking & food, family, lifestyle & culture",little chefs compete to make unforgettable mea...,2.0,seasons,hulu,3.58
11910,11910,hs793,s793,tv show,younger,NaN,NaN,united states,2021-04-15,2015,tv-14,"comedy, drama, romance",40-year-old liza (sutton foster) passes hersel...,7.0,seasons,hulu,3.49
14310,14310,ns120,s120,movie,here and there,jp habac,"janine gutierrez, jc santos, victor anastacio,...",NaN,2021-09-02,2020,tv-ma,"dramas, international movies, romantic movies",after meeting through a heated exchange on soc...,99.0,min,netflix,3.53
22549,22549,ns8359,s8359,movie,the incident,isaac ezban,"raúl méndez, nailea norvind, hernán mendoza, h...",mexico,2017-06-19,2014,tv-ma,"international movies, sci-fi & fantasy, thrillers","two criminals race down a stairwell, a cop clo...",100.0,min,netflix,3.58


In [53]:
df= df.fillna(0)
df= df.astype(str)


In [54]:
df.head()

,index,id,show_id,type,title,director,cast,country,date_added,release_year,rating_x,listed_in,description,duration_int,duration_type,Plataforma,score_mean
3854,3854,as3855,s3855,movie,happy birthday,casey tebo,"tristin mays, vanessa lengies, erik palladino",0,0,2016,r,"comedy, suspense",when two friends embark on a journey to mexica...,90.0,min,amazon,3.57
13542,13542,hs2425,s2425,tv show,chopped junior,0,0,united states,2018-09-13,2015,tv-g,"cooking & food, family, lifestyle & culture",little chefs compete to make unforgettable mea...,2.0,seasons,hulu,3.58
11910,11910,hs793,s793,tv show,younger,0,0,united states,2021-04-15,2015,tv-14,"comedy, drama, romance",40-year-old liza (sutton foster) passes hersel...,7.0,seasons,hulu,3.49
14310,14310,ns120,s120,movie,here and there,jp habac,"janine gutierrez, jc santos, victor anastacio,...",0,2021-09-02,2020,tv-ma,"dramas, international movies, romantic movies",after meeting through a heated exchange on soc...,99.0,min,netflix,3.53
22549,22549,ns8359,s8359,movie,the incident,isaac ezban,"raúl méndez, nailea norvind, hernán mendoza, h...",mexico,2017-06-19,2014,tv-ma,"international movies, sci-fi & fantasy, thrillers","two criminals race down a stairwell, a cop clo...",100.0,min,netflix,3.58


In [55]:
df = df[["title", "score_mean"]]
df= df.reset_index()
vectorizer = CountVectorizer()
title_matrix = vectorizer.fit_transform(df["title"])
cosine_sim = cosine_similarity(title_matrix)

In [56]:
with open('cosine_sim.pickle', 'wb') as f:
    pickle.dump(cosine_sim, f)

In [57]:
def get_recommendation(title: str, n: int = 5):
    # Obtener el índice de la película dada
    #indices = df[df["title"].str.contains(title, case=False)].index.tolist()
    indices = df[df['title'] == title].index[0]
    # Obtener los puntajes de similitud para la película dada
    sim_scores = list(enumerate(cosine_sim[indices]))
    #sim_scores = [(i, cosine_sim[i]) for i in indices]
    # Ordenar las películas por puntaje de similitud
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    # Obtener los índices de las películas recomendadas
    #movie_indices = [i[0] for i in sim_scores[1:n+1]]
    # Obtener las películas recomendadas
    #top_recommendations = df.iloc[movie_indices]["title"].values.tolist()
    top_recommendations = [(df.iloc[i[0]].title, df.iloc[i[0]].score_mean) for i in sim_scores[1:6]]
    return {"title": title, "recommendations": top_recommendations}

In [47]:
df.head()

,title,score_mean
0,happy birthday,3.57
1,chopped junior,3.58
2,younger,3.49
3,here and there,3.53
4,the incident,3.58


In [58]:
get_recommendation('here and there')

{'title': 'here and there',
 'recommendations': [('out there', '3.55'),
  ('freaks and geeks', '3.51'),
  ('kongsuni and friends', '3.53'),
  ('alive and kicking', '3.54'),
  ('borsalino and co.', '3.51')]}